In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
import qa_questions
import random
import joblib
from tqdm import tqdm
import feature_spaces
fit_encoding = __import__('01_fit_encoding')
import encoding_utils

### Look at examples

In [ ]:
class A:
    use_test_setup = False
    subject = 'UTS03'


args = A()

In [ ]:
story_names_train, story_names_test = fit_encoding.get_story_names(args)

In [ ]:
wordseqs = feature_spaces.get_story_wordseqs(story_names_train)
# chunks = wordseqs['theclosetthatateeverything'].chunks()
# chunk_lens = [len(chunk) for chunk in chunks]

In [ ]:
seed = 43  # 42
ngrams_examples = []
ngram_size = 10
num_examples_per_story = 1
random.seed(seed)
np.random.seed(seed)
for story_name in story_names_train:
    words_list = wordseqs[story_name].data
    ngrams_list = feature_spaces._get_ngrams_list_from_words_list(
        words_list, ngram_size=ngram_size)[ngram_size + 2:]
    ngrams_examples += np.random.choice(ngrams_list,
                                        num_examples_per_story).tolist()

In [ ]:
print('\n'.join(['- ' + ngram for ngram in ngrams_examples]))

### Look at alignment

In [ ]:
wordseqs.keys()

In [ ]:
wordseqs = feature_spaces.get_story_wordseqs(['sloth'])
# chunks = wordseqs['theclosetthatateeverything'].chunks()
# chunk_lens = [len(chunk) for chunk in chunks]

In [ ]:
ws = wordseqs['sloth']

In [ ]:
len(ws.data_times)

In [ ]:
ws.data_times

In [65]:
from typing import List


def _get_ngrams_list_from_words_list_and_times(words_list: List[str], times_list: np.ndarray[float], sec_offset: float = 4) -> List[str]:

    words_arr = np.array(words_list)
    ngrams_list = []
    for i in range(len(times_list)):
        t = times_list[i]
        t_off = t - sec_offset
        idxs = np.where(np.logical_and(
            times_list >= t_off, times_list <= t))[0]
        ngrams_list.append(' '.join(words_arr[idxs]))
    return ngrams_list


ngrams_list = _get_ngrams_list_from_words_list_and_times(
    ws.data, ws.data_times, sec_offset=3)

In [69]:
ws.tr_times

array([ -9.,  -7.,  -5.,  -3.,  -1.,   1.,   3.,   5.,   7.,   9.,  11.,
        13.,  15.,  17.,  19.,  21.,  23.,  25.,  27.,  29.,  31.,  33.,
        35.,  37.,  39.,  41.,  43.,  45.,  47.,  49.,  51.,  53.,  55.,
        57.,  59.,  61.,  63.,  65.,  67.,  69.,  71.,  73.,  75.,  77.,
        79.,  81.,  83.,  85.,  87.,  89.,  91.,  93.,  95.,  97.,  99.,
       101., 103., 105., 107., 109., 111., 113., 115., 117., 119., 121.,
       123., 125., 127., 129., 131., 133., 135., 137., 139., 141., 143.,
       145., 147., 149., 151., 153., 155., 157., 159., 161., 163., 165.,
       167., 169., 171., 173., 175., 177., 179., 181., 183., 185., 187.,
       189., 191., 193., 195., 197., 199., 201., 203., 205., 207., 209.,
       211., 213., 215., 217., 219., 221., 223., 225., 227., 229., 231.,
       233., 235., 237., 239., 241., 243., 245., 247., 249., 251., 253.,
       255., 257., 259., 261., 263., 265., 267., 269., 271., 273., 275.,
       277., 279., 281., 283., 285., 287., 289., 29

In [ ]:
ws.data

In [ ]:
from typing import List
    """Concatenate running list of words into grams with spaces in between
    """
    ngrams_list = []
    for i in range(len(words_list)):
        l = max(0, i - ngram_size)
        ngram = ' '.join(words_list[l: i + 1])
        ngrams_list.append(ngram.strip())
    return ngrams_list

In [ ]:
len(ws.chunks())

In [ ]:
len(feature_spaces._get_ngrams_list_from_chunks(ws.chunks()))

In [ ]:
resp = encoding_utils.get_response(['sloth'], args.subject)

In [ ]:
resp.shape

In [ ]:
ws = wo

In [ ]:
len(wordseqs['wheretheressmoke'].data)

In [ ]:
len(wordseqs['wheretheressmoke'].chunks()[10:-5])

In [ ]:
wordseqs['wheretheressmoke'].chunks()


def get_ngrams_list_from_chunks(chunks, num_trs=2):
    ngrams_list = []
    for i, chunk in enumerate(chunks):
        # print(chunks[i - num_trs:i])
        # sum(chunks[i - num_trs:i], [])
        chunk_block = chunks[i - num_trs:i]
        if len(chunk_block) == 0:
            ngrams_list.append('')
        else:
            chunk_block = np.concatenate(chunk_block)
            ngrams_list.append(' '.join(chunk_block))

    return ngrams_list


ngrams_list = get_ngrams_list_from_chunks(
    wordseqs['wheretheressmoke'].chunks(), num_trs=2)

In [ ]:
ngrams_list

In [ ]:
args

### Load fitted models

In [ ]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/results_mar30'
experiment_filename = '../experiments/01_fit_encoding.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)
# imodelsx.process_results.delete_runs_in_dataframe(
# r[r.feature_space == 'qa_embedder-10'], actually_delete=True)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)

In [ ]:
args = r[(r.feature_space == 'qa_embedder-5') * (r.pc_components == 100) * (r.ndelays == 4)
         ].sort_values(by='corrs_tune_mean', ascending=False).iloc[0]

In [ ]:
args['corrs_test_pc']

In [ ]:
story_names_train, story_names_test = fit_encoding.get_story_names(args)
stim_test_delayed, resp_test = fit_encoding.get_data(args, story_names_test)
stim_train_delayed, resp_train = fit_encoding.get_data(args, story_names_train)